In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip -q install huggingface chromadb sentence-transformers

In [ ]:
!pip -q install configparser langchain
!pip -q install transformers huggingface_hub 

In [ ]:
!pip -q install openai pinecone-client

In [56]:
!pip -q install pypdf unstructured[local-inference]

Above are various libraries required for this notebook.

In [9]:
import configparser

apiData = configparser.ConfigParser()
apiData.read_file(open('/content/apidata.config'))

In [6]:
import os
os.environ['OPENAI_API_KEY'] = apiData["OPENAI"]["KEY"]

In [19]:
from langchain.llms import OpenAI

fact_llm = OpenAI(temperature=0)

In [11]:
#initiate pinecone client

import pinecone

pinecone.init(api_key = apiData['OPENAI']['PINE'],
              environment= apiData['OPENAI']['PINEVIR'])

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
pinecone.create_index("test-index",dimension=1536)

In [58]:
pinecone.list_indexes()

['test-index']

In [59]:
pinecone.describe_index("test-index")

IndexDescription(name='test-index', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [18]:
pinecone.delete_index("your-index")

In [21]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader

In [22]:
import openai

embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In [25]:
type(res)

openai.openai_object.OpenAIObject

In [26]:
len(res['data'])

2

In [27]:
len(res['data'][0]['embedding'])

1536

In [28]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

In [31]:
res['usage']

<OpenAIObject at 0x7f461af22450> JSON: {
  "prompt_tokens": 13,
  "total_tokens": 13
}

In [35]:
test_file = PyPDFLoader("/content/test_pdf.pdf")

In [39]:
file_split = test_file.load_and_split()

In [ ]:
file_split

In [46]:
unstructure_file = UnstructuredPDFLoader("/content/test_pdf.pdf",
                                         mode="elements")
data_file = unstructure_file.load()

In [ ]:
data_file

In [48]:
from langchain.embeddings import OpenAIEmbeddings
embed = OpenAIEmbeddings()

In [60]:
index_name = 'test-index'

if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
else:
  print("Index aleady exists")

Index aleady exists


In [51]:
from langchain.vectorstores import Pinecone

test_cone = Pinecone.from_documents(documents=data_file,
                                     embedding=embed,
                                    index_name=index_name)

In [ ]:
# Starting the custom tool route

In [53]:
test_retriever =RetrievalQA.from_chain_type(llm=fact_llm, 
                                           chain_type="stuff", 
                                           retriever=test_cone.as_retriever())

In [55]:
test_retriever.run("What are agents in Langchain?")

' Agents in Langchain are pieces of code that can be used to create chains, which are sequences of instructions that can be used to automate tasks. Agents can also use memory to persist state between calls.'

In [62]:
reload_cone = pinecone.Index("test-index")

In [71]:
reload_cone.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 55}},
 'total_vector_count': 55}

In [77]:
fetch_id1 = reload_cone.fetch(['9b879795-34ee-4b0e-997b-04ca91e48562'])

In [ ]:
pinecone.whoami()
#WhoAmIResponse(username='your_uname', user_label='default', projectname='your_pjt')

In [64]:
reload_docs = Pinecone.from_existing_index("test-index",
                                           embedding=embed)

In [65]:
reload_docs.as_retriever()

VectorStoreRetriever(vectorstore=<langchain.vectorstores.pinecone.Pinecone object at 0x7f45d50c8130>, search_type='similarity', search_kwargs={})

In [68]:
get_match = reload_cone.query(
  vector=[0.3 for x in range(0,1536)],
  top_k=3,
  include_values=True
)

In [ ]:
get_match['matches'][0]

In [79]:
reload_docs.similarity_search("How to use Agents?")

[Document(page_content='Agents:', metadata={'category': 'Title', 'filename': '/content/test_pdf.pdf', 'page_number': 2.0, 'source': '/content/test_pdf.pdf'}),
 Document(page_content='Agents', metadata={'category': 'Title', 'filename': '/content/test_pdf.pdf', 'page_number': 1.0, 'source': '/content/test_pdf.pdf'}),
 Document(page_content='end to end agents.', metadata={'category': 'NarrativeText', 'filename': '/content/test_pdf.pdf', 'page_number': 2.0, 'source': '/content/test_pdf.pdf'}),
 Document(page_content='standard interface for agents, a selection of agents to choose from, and examples of', metadata={'category': 'NarrativeText', 'filename': '/content/test_pdf.pdf', 'page_number': 2.0, 'source': '/content/test_pdf.pdf'}),
 Document(page_content='examples of chains/agents that use memory.', metadata={'category': 'NarrativeText', 'filename': '/content/test_pdf.pdf', 'page_number': 2.0, 'source': '/content/test_pdf.pdf'})]